In [1]:
import json
import numpy as np
import pandas as pd
import requests

from pathlib import Path


def resolve(filename):
    return 'https://cdn.thegamesdb.net/images/original/' + filename


def save(game, url):
    print(f"Fetching: {url}")

    filename = f"{game}.png"
    path = Path('images/boxart')
    location = path.joinpath(filename)

    if location.is_file():
        return

    image = requests.get(url)

    with open(location, 'wb') as handle:
        if image.status_code == 200:
            handle.write(image.content)
        else:
            print('Error')


def get_games(file):
    games = file.get('data').get('games')
    dataframe = pd.DataFrame(games)

    # SNES: 6, NES:  7

    mask = (
        (dataframe.platform == 6) |
        (dataframe.platform == 7)
    )

    drop = [
        'alternates',
        'coop',
        'hashes',
        'players',
        'uids',
        'youtube'
    ]

    dataframe = dataframe[mask]
    dataframe = dataframe.drop(drop, axis=1)

    return dataframe


def get_boxart(file):
    boxart = file.get('include').get('boxart')
    _ = boxart.get('base_url').get('original')
    images = boxart.get('data')

    boxart = []

    for key, value in images.items():
        for v in value:
            condition = (
                v.get('type') == 'boxart' and
                v.get('side') == 'front'
            )

            if condition:
                v['id'] = int(key)
                boxart.append(v)

    drop = [
        'side',
        'type'
    ]

    dataframe = pd.DataFrame(boxart)
    dataframe = dataframe.drop(drop, axis=1)

    return dataframe


def get_filename(file):
    print(int(file.stem))
    return int(file.stem)

In [2]:
path = Path('images/boxart')
path.mkdir(parents=True, exist_ok=True)

with open('data/database.json', 'r') as handle:
    file = json.load(handle)

    games = get_games(file)
    boxart = get_boxart(file)

dataframe = games.merge(
    boxart,
    how='inner',
    on='id'
)

# gid = dataframe.id.to_numpy()

filelist = {
    int(file.stem): file.as_posix()
    for file in path.glob('*')
    if file.is_file()
}

image = pd.DataFrame([filelist])
image = image.transpose()

dataframe = pd.concat(
    [dataframe, image]
)

In [3]:
dataframe

,id,game_title,release_date,platform,region_id,country_id,overview,rating,developers,genres,publishers,filename,resolution,0
0,5.0,Donkey Kong,1986-06-01,7.0,2.0,50.0,Can you save Mario&#039;s girl from the clutch...,E - Everyone,[6053],[15],[3],boxart/front/5-2.jpg,1534x2100,NaN
1,41.0,Super Mario Kart,1992-09-01,6.0,0.0,0.0,The Super Mario GoKart Park is open for tons o...,E - Everyone,[6037],[7],[3],boxart/front/41-1.jpg,2100x1534,NaN
2,76.0,Bionic Commando,1988-12-21,7.0,0.0,0.0,Bionic Commando takes place ten years after an...,E - Everyone,[1436],"[1, 15]",[9],boxart/front/76-1.jpg,None,NaN
3,83.0,Final Fantasy III,1994-10-11,6.0,0.0,0.0,MagiTek has been reborn. And the end of the wo...,T - Teen,[8096],[4],[11],boxart/front/83-1.jpg,2100x1536,NaN
4,112.0,Super Mario Bros. 3,1990-02-12,7.0,0.0,0.0,"Fight monsters and mini-bosses, avoid ghosts a...",E - Everyone,[6055],[15],[3],boxart/front/112-2.jpg,1534x2100,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,images/boxart/99965.png
99966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,images/boxart/99966.png
99968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,images/boxart/99968.png
99984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,images/boxart/99984.png
